

# **Laboratorio 11: Pienso, luego predigo 💡**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Diego Acuña
- Nombre de alumno 2: Tomás Ramírez

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/Diego-Acuna/mds-laboratorios)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [1]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [2]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

El ambiente **Blackjack** de Gymnasium modela una versión simplificada del juego de Blackjack como un **proceso de decisión de Markov (MDP)**:

- **Estados (S)**  
  Cada estado se representa típicamente como una tupla $(\text{sum\_jugador}, \text{carta\_visible\_dealer}, \text{tiene\_as\_usable})$, donde:
  - `sum_jugador`: suma actual de las cartas del jugador.
  - `carta_visible_dealer`: valor (1–10) de la carta que muestra el dealer.
  - `tiene_as_usable`: booleano que indica si el jugador tiene un As que puede contar como 11 sin pasarse de 21.  
  Además, hay estados terminales en los que la mano ya terminó (el episodio se acaba).

- **Acciones (A)**  
  Normalmente son solo dos:
  - `Hit` (pedir carta): el jugador solicita una nueva carta.
  - `Stick` (plantarse): el jugador deja de pedir y le toca jugar al dealer (según la política fija del ambiente).

- **Recompensas (R)**  
  Se entregan al final del episodio (cuando el jugador se pasa, se planta y termina la mano, etc.):
  - **+1** si el jugador gana.
  - **0** si hay empate (*push*).
  - **−1** si el jugador pierde.  

Así, dado un estado (suma actual, carta del dealer, As usable) y una acción (`hit` o `stick`), el ambiente transiciona aleatoriamente a un nuevo estado según las cartas que salen del mazo y al final entrega una recompensa según el resultado de la mano.


#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

In [3]:
n_episodes = 5000
returns = []

for ep in range(n_episodes):
    obs, info = env.reset()
    done = False
    ep_return = 0.0

    while not done:
        action = env.action_space.sample()

        obs, reward, terminated, truncated, info = env.step(action)
        ep_return += reward
        done = terminated or truncated

    returns.append(ep_return)

returns = np.array(returns)

promedio = returns.mean()
desviacion = returns.std()

print(f"Número de episodios: {n_episodes}")
print(f"Retorno promedio: {promedio:.4f}")
print(f"Desviación estándar del retorno: {desviacion:.4f}")


Número de episodios: 5000
Retorno promedio: -0.3890
Desviación estándar del retorno: 0.8976


Con los resultados obtenidos, al usar una política completamente aleatoria en el ambiente de Blackjack y simular $5000$ episodios, se observa un retorno promedio de aproximadamente $-0.3890$ con una desviación estándar cercana a $0.8976$. Esto indica que, en promedio, la política pierde alrededor de $0.39$ unidades por mano, lo que es coherente con el hecho de que elegir acciones al azar en Blackjack es claramente desventajoso. Dado que las recompensas típicas son $-1$ cuando se pierde, $0$ cuando hay empate y $+1$ cuando se gana, un valor esperado negativo muestra que se pierden más partidas de las que se ganan.

La desviación estándar relativamente alta refleja la variabilidad natural del juego: hay episodios ganados, perdidos y empatados, por lo que las recompensas individuales fluctúan bastante alrededor del promedio. Sin embargo, al considerar un número grande de episodios, el retorno medio negativo permite concluir que esta política aleatoria constituye un baseline pobre, que sirve como referencia mínima. Cualquier agente entrenado posteriormente debiese superar este rendimiento (acercarse a 0 o incluso obtener un promedio positivo) para considerarse efectivo en comparación con “jugar al azar”.

Se concluye que el performance es malo pues el promedio es negativo. Además, las recompensas obtenidas reflejan que, al jugar al azar, el agente básicamente no explota ninguna estructura del juego, por lo que ganar es casi cuestión de suerte.

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

In [4]:
from stable_baselines3 import PPO

# Entrenamos un agente PPO para Blackjack
model_blackjack = PPO(
    "MlpPolicy",
    env,
    verbose=1
)

# Número de pasos de entrenamiento
total_timesteps = 200_000
model_blackjack.learn(total_timesteps=total_timesteps)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.33     |
|    ep_rew_mean     | -0.42    |
| time/              |          |
|    fps             | 1727     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.33     |
|    ep_rew_mean     | -0.42    |
| time/              |          |
|    fps             | 1727     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.25        |
|    ep_rew_mean          | -0.41       |
| time/                   |             |
|    fps        

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [5]:
def evaluate_agent(model, env, n_episodes=5000):
    returns = []

    for ep in range(n_episodes):
        obs, info = env.reset()
        done = False
        ep_return = 0.0

        while not done:
            # Acción dada por el agente entrenado
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, info = env.step(action)
            ep_return += reward
            done = terminated or truncated

        returns.append(ep_return)

    returns = np.array(returns)
    print(f"Número de episodios: {n_episodes}")
    print(f"Retorno promedio: {returns.mean():.4f}")
    print(f"Desviación estándar del retorno: {returns.std():.4f}")
    return returns

# Evaluar el modelo entrenado
returns_trained = evaluate_agent(model_blackjack, env, n_episodes=5000)


Número de episodios: 5000
Retorno promedio: -0.0290
Desviación estándar del retorno: 0.9557


Repitiendo el experimento de la pregunta 1.1.2, pero ahora utilizando el modelo entrenado, se obtiene un retorno promedio de aproximadamente $-0.0290$ con una desviación estándar cercana a $0.9557$ en 5000 episodios. En comparación con el baseline aleatorio, cuyo retorno promedio era alrededor de $-0.3890$, se observa una mejora clara: el agente entrenado pierde mucho menos en promedio y su valor esperado está bastante más cerca de $0$.

Aunque el retorno sigue siendo levemente negativo, el hecho de pasar de $-0.3890$ a $-0.0290$ indica que el agente ha aprendido una política significativamente mejor que la aleatoria, reduciendo la frecuencia o el impacto de las pérdidas. La desviación estándar similar ($\approx 0.95$) muestra que la variabilidad de las recompensas por episodio se mantiene alta, algo esperable en Blackjack por la naturaleza aleatoria del juego, pero el desplazamiento del promedio hacia $0$ permite concluir que el performance del modelo entrenado es claramente superior al escenario baseline.

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

In [6]:
def agent_action_from_state(model, state):
    """
    state: puede ser un tuple como (player_sum, dealer_card, usable_ace)
           o un np.array (depende de cómo esté definido el env).
    """
    if isinstance(state, tuple):
        state = np.array(state, dtype=np.float32)

    action, _ = model.predict(state, deterministic=True)
    return int(action)  # 0 = stick, 1 = hit en Blackjack-v1

# Escenario 1:
# Suma del agente = 6, dealer muestra 7, sin As usable
state1 = (6, 7, False)

# Escenario 2:
# Suma del agente = 19, dealer muestra 3, con As usable
state2 = (19, 3, True)

action1 = agent_action_from_state(model_blackjack, state1)
action2 = agent_action_from_state(model_blackjack, state2)

action_meaning = {0: "STICK (quedarse)", 1: "HIT (pedir carta)"}

print("Estado 1 (6, dealer 7, sin As usable):", action1, "->", action_meaning[action1])
print("Estado 2 (19, dealer 3, con As usable):", action2, "->", action_meaning[action2])


Estado 1 (6, dealer 7, sin As usable): 1 -> HIT (pedir carta)
Estado 2 (19, dealer 3, con As usable): 0 -> STICK (quedarse)


Para esta parte se construyó una función que, dado un estado del ambiente (tupla con $(\text{suma\_jugador},\ \text{carta\_dealer},\ \text{as\_usable})$), lo pasa por el modelo entrenado y devuelve la acción elegida por el agente. Usando esa función en los dos escenarios indicados, se obtuvo:

- Estado 1: suma del jugador $= 6$, dealer muestra un $7$, sin As usable  
  Acción del agente: `1` $\Rightarrow$ **HIT** (pedir carta).

- Estado 2: suma del jugador $= 19$, dealer muestra un $3$, con As usable (soft 19)  
  Acción del agente: `0` $\Rightarrow$ **STICK** (quedarse).

Estas decisiones son coherentes con las reglas y con la estrategia básica de Blackjack.  

En el primer caso, con una suma de $6$ contra un $7$ del dealer, la mano del jugador es muy débil; lo razonable es seguir pidiendo cartas, por lo que que el agente elija HIT es consistente con una política sensata. Básicamente hay que pedir si o si pues sea lo que sea que salga en la siguiente carta es imposible pasarse de $21$.

En el segundo caso, con un “soft $19$” ($19$ con As usable) frente a un $3$ del dealer, la estrategia básica suele indicar plantarse (o doblar si esa acción existe; en este ambiente sólo están disponibles HIT y STICK). En consecuencia, que el agente elija STICK en este estado también es coherente con una política de juego razonable. (Igual yo hitearía con un $19$ teniendo un As XD).


### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [7]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

c:\Users\samot\AppData\Local\Programs\Python\Python312\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [8]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

El ambiente **LunarLander** (en su versión continua) modela el problema de aterrizar suavemente una nave en una superficie lunar como un **proceso de decisión de Markov (MDP)**. El estado $S$ se representa mediante un vector continuo que describe la situación física de la nave: posición $(x, y)$ relativa a la zona de aterrizaje, velocidades $(\dot x, \dot y)$, ángulo de la nave $\theta$, velocidad angular $\dot \theta$ y dos variables binarias que indican si cada una de las patas está en contacto con el suelo. Este estado resume toda la información relevante para decidir la acción siguiente.

Las acciones $A$ en la versión continua (con `continuous = True`) son **valores continuos** que controlan la intensidad de los motores: típicamente un par de números reales que indican la fuerza del motor principal (vertical) y de los motores laterales (para inclinar la nave). Es decir, en cada paso el agente elige un vector de acción en vez de una acción discreta fija. Las recompensas $R$ están diseñadas para favorecer un aterrizaje suave en la plataforma: se otorgan recompensas positivas por acercarse al punto de aterrizaje, reducir la velocidad y tocar el suelo de forma estable con las dos patas, y recompensas negativas por chocar, salir de la zona o gastar combustible en exceso.

En comparación con el ambiente de `Blackjack`, donde las acciones son **discretas y muy pocas** (por ejemplo, “pedir carta” o “plantarse”), en `LunarLander` continuo las acciones forman un **espacio de acciones continuo**: el agente no sólo elige “qué hacer”, sino también “cuánto” empuje aplicar en cada motor. Esto hace que el espacio de acciones sea mucho más rico y que el control sea de tipo continuo en lugar de decisiones discretas simples.


#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

In [9]:
def evaluate_random_policy(env, n_episodes):
    """
    Baseline con política aleatoria para LunarLander (continuous=True).
    """
    returns = []

    for ep in range(n_episodes):
        obs, _ = env.reset()
        done = False
        ep_return = 0.0

        while not done:
            # Acción completamente aleatoria en el espacio continuo
            action = env.action_space.sample()
            _, reward, terminated, truncated, _ = env.step(action)
            ep_return += reward
            done = terminated or truncated

        returns.append(ep_return)

    returns = np.array(returns)
    print(f"Número de episodios: {n_episodes}")
    print(f"Retorno promedio: {returns.mean():.2f}")
    print(f"Desviación estándar del retorno: {returns.std():.2f}")
    return returns

# Corremos el baseline
random_returns = evaluate_random_policy(env, n_episodes=10)

Número de episodios: 10
Retorno promedio: -302.39
Desviación estándar del retorno: 126.51


Con los resultados obtenidos, al utilizar una política completamente aleatoria en el ambiente `LunarLander` (versión continua) y simular 10 episodios, se observa un retorno promedio de aproximadamente $-302.39$ con una desviación estándar cercana a $126.51$. Este valor promedio tan negativo indica que la política aleatoria tiene un desempeño claramente deficiente: en general, el agente no logra realizar aterrizajes suaves, suele chocar, desviarse de la zona objetivo o gastar energía de manera ineficiente. Dado que en `LunarLander` las recompensas se construyen para premiar aterrizajes estables y penalizar colisiones o maniobras malas, un retorno esperado en torno a $-300$ refleja un comportamiento muy lejos de lo deseable.

La desviación estándar relativamente alta sugiere que hay variabilidad entre episodios (en algunos intentos el agente puede caer “menos mal” que en otros), pero todos los resultados se concentran en un rango claramente negativo. En consecuencia, esta política aleatoria sirve como un baseline pobre: establece un piso de rendimiento que cualquier agente entrenado debería ser capaz de superar de forma consistente para considerarse mínimamente competente en la tarea de aterrizar la nave.


#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

In [10]:
# Definimos un modelo PPO para LunarLander continuo
model_lunar = PPO(
    "MlpPolicy",
    env,
    verbose=1,
)

# Entrenamos el modelo para 10000 timesteps
model_lunar.learn(total_timesteps=10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 107      |
|    ep_rew_mean     | -219     |
| time/              |          |
|    fps             | 1763     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 107      |
|    ep_rew_mean     | -219     |
| time/              |          |
|    fps             | 1763     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 113         |
|    ep_rew_mean          | -225        |
| time/                   |             |
|    fps        

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

In [11]:
def evaluate_agent(model, env, n_episodes):
    """
    Evalúa un agente entrenado en LunarLander.
    """
    returns = []

    for ep in range(n_episodes):
        obs, _ = env.reset()
        done = False
        ep_return = 0.0

        while not done:
            # Política del agente entrenado
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)
            ep_return += reward
            done = terminated or truncated

        returns.append(ep_return)

    returns = np.array(returns)
    print(f"Número de episodios: {n_episodes}")
    print(f"Retorno promedio: {returns.mean():.2f}")
    print(f"Desviación estándar del retorno: {returns.std():.2f}")
    return returns

# Evaluamos el modelo entrenado
trained_returns = evaluate_agent(model_lunar, env, n_episodes=10)

Número de episodios: 10
Retorno promedio: -229.24
Desviación estándar del retorno: 86.36


Repitiendo el experimento de la pregunta 1.2.2, pero utilizando el modelo entrenado, se obtiene un retorno promedio de aproximadamente $-229.24$ con una desviación estándar cercana a $86.36$ en 10 episodios. En comparación con el baseline aleatorio, cuyo retorno promedio era alrededor de $-302.39$, se observa una mejora: el agente entrenado sigue teniendo un desempeño negativo, pero pierde menos recompensa en promedio y se acerca a un comportamiento algo más razonable en la tarea de aterrizar la nave.

Además, la desviación estándar disminuye desde aproximadamente $126.51$ a $86.36$, lo que sugiere que el comportamiento del agente se vuelve algo más consistente entre episodios (menos variabilidad extrema). Sin embargo, el hecho de que el retorno esperado siga siendo claramente negativo indica que, aunque el modelo entrenado supera al baseline aleatorio, todavía está lejos de un desempeño satisfactorio: el agente no logra aterrizajes buenos de manera sistemática y aún comete errores importantes en la mayoría de los episodios.


#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

In [12]:
# Definimos un modelo PPO optimizado para LunarLander continuo
model_lunar_opt = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4, # Modificamos el learning rate
    batch_size=64, # Modificamos el tamaño del batch
)

# Entrenamos el modelo optimizado
model_lunar_opt.learn(total_timesteps=500000)  # Modificamos el número de timesteps

# Evaluamos el modelo optimizado
evaluate_agent(model_lunar_opt, env, n_episodes=10)

Número de episodios: 10
Retorno promedio: 204.39
Desviación estándar del retorno: 93.10


array([268.72731177, 242.08270975, 203.05854448, 232.5539607 ,
       227.37086614, 250.79172504, 230.35210321, -70.45022041,
       238.33583518, 221.09885924])

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [29]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [30]:
%pip install --upgrade --quiet PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [31]:
import PyPDF2
from pathlib import Path

DOCS_DIR = Path("docs")

doc_paths = [str(p) for p in DOCS_DIR.glob("*.pdf")]

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [32]:
from langchain_huggingface import HuggingFaceEmbeddings
import PyPDF2
import numpy as np

# Leemos el texto de los PDFs
texts = []
for path in doc_paths:
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        pages_text = []
        for page in reader.pages:
            page_text = page.extract_text() or ""
            pages_text.append(page_text)
        full_text = "\n".join(pages_text)
        if full_text.strip():
            texts.append(full_text)

print(f"Se cargaron {len(texts)} documentos de texto a partir de {len(doc_paths)} PDFs.")

# Definimos una función para dividir el texto en chunks con overlap
def split_text(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

# Generamos los chunks de texto para todos los documentos
text_chunks = []
for t in texts:
    text_chunks.extend(split_text(t, chunk_size=1000, overlap=200))

print(f"Se generaron {len(text_chunks)} chunks de texto.")

# Modelo de embeddings multilingüe (el que se dio en el foro)
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# Generamos los embeddings para los chunks de texto
chunk_embeddings = embeddings_model.embed_documents(text_chunks)
emb_matrix = np.array(chunk_embeddings)

# Mostramos la forma de la matriz de embeddings
print(f"Dimensiones de la matriz de embeddings: {emb_matrix.shape}")


Se cargaron 4 documentos de texto a partir de 4 PDFs.
Se generaron 1328 chunks de texto.
Dimensiones de la matriz de embeddings: (1328, 384)
Dimensiones de la matriz de embeddings: (1328, 384)


#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [33]:
from google import genai

# Creamos el cliente de Gemini usando la API key
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

def retrieve_relevant_chunks(question: str, k: int = 5):
    """
    Dada una pregunta, retorna los k chunks más similares usando
    similitud de coseno entre embeddings de HuggingFace.
    """
    # Embedding de la pregunta
    query_vec = np.array(embeddings_model.embed_query(question))

    # Similaridad de coseno con todos los chunks
    doc_norms = np.linalg.norm(emb_matrix, axis=1)
    q_norm = np.linalg.norm(query_vec) + 1e-10
    sims = (emb_matrix @ query_vec) / (doc_norms * q_norm + 1e-10)

    # Índices de los k más similares (ordenados de mayor a menor similitud)
    top_k_idx = np.argsort(sims)[-k:][::-1]

    # Retornamos los textos correspondientes
    return [text_chunks[i] for i in top_k_idx]

def rag_chain(question: str, k: int = 5) -> str:
    """
    Chain de RAG: dado una pregunta, recupera contexto de los PDFs
    y obtiene una respuesta del LLM de Google.
    """
    relevant_chunks = retrieve_relevant_chunks(question, k=k)
    context = "\n\n".join(relevant_chunks)

    system_instruction = (
        "Eres un asistente que responde preguntas usando exclusivamente "
        "el contexto proporcionado. Si la información no está en el contexto, "
        "di explícitamente que no tienes información suficiente."
    )

    contents = [
        system_instruction,
        f"Contexto:\n{context}",
        f"Pregunta: {question}",
        "Respuesta (en español):",
    ]

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents,
    )

    return response.text.strip()

# Guardamos la chain RAG en una variable
chain = rag_chain


#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [46]:
# Definimos algunas preguntas de prueba y sus respuestas correctas esperadas
qa_pairs = [
    (
        "¿Cómo se define una matriz A de m filas y n columnas con coeficientes en un cuerpo K?",
        "Una matriz A es una tabla de doble entrada A = (a_ij) con m filas y n columnas, "
        "donde a_ij ∈ K para todo i = 1,…,m y j = 1,…,n."
    ),
    (
        "¿Cuál es la definición de subsucesión de una sucesión (s_n)?",
        "Sea (s_n) una sucesión y φ: N → N una función estrictamente creciente; "
        "la subsucesión generada por φ es (u_n) con u_n = s_{φ(n)}."
    ),
    (
        "¿Qué significa que un conjunto sea numerable?",
        "Un conjunto es numerable si tiene la misma cardinalidad que N, es decir, "
        "si existe una biyección entre el conjunto y N."
    ),
]

# Probamos la chain RAG con las preguntas definidas y mostramos las respuestas en pantalla
for i, (question, expected_answer) in enumerate(qa_pairs, start=1):
    print(f"\n=== Pregunta {i} ===")
    print("Pregunta:", question)
    print("Respuesta correcta esperada:\n ", expected_answer, "\n")

    rag_answer = rag_chain(question, k=5)
    print("Respuesta de la solución RAG:\n ", rag_answer)
    
# IMPORTANTE: Acá no entiendo por qué se imprime más de una vez el for, es un bug que no pude arreglar.
# De todas maneras igual se puede ver la información relevante y responder lo pedido en base a ello.



=== Pregunta 1 ===
Pregunta: ¿Cómo se define una matriz A de m filas y n columnas con coeficientes en un cuerpo K?
Respuesta correcta esperada:
  Una matriz A es una tabla de doble entrada A = (a_ij) con m filas y n columnas, donde a_ij ∈ K para todo i = 1,…,m y j = 1,…,n. 

Respuesta de la solución RAG:
  Una matriz A de m filas y n columnas con coeficientes en un cuerpo K se nota como A = (aij) y pertenece al conjunto Mmn(K), que es el conjunto de todas las matrices de m filas y n columnas con coeficientes en el cuerpo K.

=== Pregunta 2 ===
Pregunta: ¿Cuál es la definición de subsucesión de una sucesión (s_n)?
Respuesta correcta esperada:
  Sea (s_n) una sucesión y φ: N → N una función estrictamente creciente; la subsucesión generada por φ es (u_n) con u_n = s_{φ(n)}. 

Respuesta de la solución RAG:
  Una matriz A de m filas y n columnas con coeficientes en un cuerpo K se nota como A = (aij) y pertenece al conjunto Mmn(K), que es el conjunto de todas las matrices de m filas y n col

El RAG si devolvió las respuestas esperadas.

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [35]:
import re

# Usamos una pregunta más difícil que las de 2.1.4 para que pueda explayarse más
analysis_question = "¿Cómo puedo diagonalizar una matriz?"

def build_index_for_chunk_size(chunk_size, overlap=200):
    """
    Reconstruye los chunks y sus embeddings para un tamaño de chunk dado,
    usando los textos originales `texts` y el mismo modelo de embeddings.
    """
    local_chunks = []
    for t in texts:
        local_chunks.extend(split_text(t, chunk_size=chunk_size, overlap=overlap))

    local_emb_matrix = np.array(embeddings_model.embed_documents(local_chunks))
    return local_chunks, local_emb_matrix


def retrieve_semantic(question, chunks_idx, emb_matrix_idx, k=5):
    """
    Búsqueda SEMÁNTICA: similitud de coseno entre embeddings.
    """
    query_vec = np.array(embeddings_model.embed_query(question))

    doc_norms = np.linalg.norm(emb_matrix_idx, axis=1)
    q_norm = np.linalg.norm(query_vec) + 1e-10
    sims = (emb_matrix_idx @ query_vec) / (doc_norms * q_norm + 1e-10)

    top_k_idx = np.argsort(sims)[-k:][::-1]
    return [chunks_idx[i] for i in top_k_idx]


def _tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-záéíóúüñ0-9\s]", " ", text)
    return text.split()


def retrieve_keyword(question, chunks_idx, k=5):
    """
    Búsqueda LÉXICA: ranking por cantidad de palabras en común (muy simple).
    No usa embeddings.
    """
    q_tokens = set(_tokenize(question))
    scores = []

    for chunk in chunks_idx:
        c_tokens = set(_tokenize(chunk))
        scores.append(len(q_tokens & c_tokens))

    scores = np.array(scores)
    top_k_idx = np.argsort(scores)[-k:][::-1]
    return [chunks_idx[i] for i in top_k_idx]


def rag_with_index(question, chunks_idx, emb_matrix_idx=None, k=5, search_type="semantic"):
    """
    Versión de RAG que permite elegir:
    - tamaño de chunk (a través del índice que se le pasa),
    - k (cantidad de chunks),
    - tipo de búsqueda: 'semantic' o 'keyword'.
    """
    if search_type == "semantic":
        assert emb_matrix_idx is not None, "Se requiere emb_matrix_idx para búsqueda semántica"
        relevant_chunks = retrieve_semantic(question, chunks_idx, emb_matrix_idx, k=k)
    elif search_type == "keyword":
        relevant_chunks = retrieve_keyword(question, chunks_idx, k=k)
    else:
        raise ValueError("search_type debe ser 'semantic' o 'keyword'")

    context = "\n\n".join(relevant_chunks)

    contents = [
        "Eres un asistente que responde preguntas usando exclusivamente el contexto proporcionado. "
        "Si la información no está en el contexto, di explícitamente que no tienes información suficiente.",
        f"Contexto:\n{context}",
        f"Pregunta: {question}",
        "Respuesta (en español):",
    ]

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents,
    )
    return response.text.strip()


# =====================================================================
# 1) EFECTO DEL TAMAÑO DEL CHUNK
# =====================================================================

print("#" * 120)
print("ANÁLISIS: TAMAÑO DEL CHUNK\n")

# Usamos varios tamaños de chunk
chunk_sizes = [500, 1000, 2000]
answers_chunk_size = {}

for cs in chunk_sizes:
    print("=" * 120)
    print(f"Pregunta: {analysis_question}")
    print(f"Tamaño de chunk: {cs}\n")

    cs_chunks, cs_embs = build_index_for_chunk_size(chunk_size=cs, overlap=200)
    ans = rag_with_index(analysis_question, cs_chunks, cs_embs, k=5, search_type="semantic")
    answers_chunk_size[cs] = ans
    print(ans, "\n")


# =====================================================================
# 2) EFECTO DE LA CANTIDAD DE CHUNKS RECUPERADOS (k)
# =====================================================================

print("#" * 120)
print("ANÁLISIS: CANTIDAD DE CHUNKS k\n")

# Usamos un tamaño de chunk fijo
base_chunks, base_embs = build_index_for_chunk_size(chunk_size=1000, overlap=200)

# Ks a probar
ks = [1, 3, 5, 10]
answers_k = {}

for k in ks:
    print("=" * 120)
    print(f"Pregunta: {analysis_question}")
    print(f"k (chunks recuperados): {k}\n")

    ans = rag_with_index(analysis_question, base_chunks, base_embs, k=k, search_type="semantic")
    answers_k[k] = ans
    print(ans, "\n")


# =====================================================================
# 3) EFECTO DEL TIPO DE BÚSQUEDA
# =====================================================================

print("#" * 120)
print("ANÁLISIS: TIPO DE BÚSQUEDA\n")

# Tipos de búsqueda a probar
search_types = ["semantic", "keyword"]
answers_search_type = {}

for st in search_types:
    print("=" * 120)
    print(f"Pregunta: {analysis_question}")
    print(f"Tipo de búsqueda: {st}\n")

    # Para búsqueda léxica no necesitamos emb_matrix_idx
    if st == "semantic":
        ans = rag_with_index(analysis_question, base_chunks, base_embs, k=5, search_type=st)
    else:
        ans = rag_with_index(analysis_question, base_chunks, None, k=5, search_type=st)

    answers_search_type[st] = ans
    print(ans, "\n")

print("#" * 120)
print("Experimentos de sensibilidad completados.")


########################################################################################################################
ANÁLISIS: TAMAÑO DEL CHUNK

Pregunta: ¿Cómo puedo diagonalizar una matriz?
Tamaño de chunk: 500

Para diagonalizar una matriz, o para obtener una matriz diagonal D de una matriz triangular superior ~A, se puede "escalonar hacia arriba la matriz ~A".

El procedimiento descrito en el contexto es el siguiente:
1.  **Obtener una matriz triangular superior ~A**: Esto se logra mediante el escalonamiento de Gauss sobre la matriz (AjI).
2.  **Hacer ceros los elementos sobre la diagonal**: Una vez que se ha obtenido la matriz triangular superior ~A, se debe "realizar el mismo procedimiento para los elementos sobre la diagonal". Esto implica operaciones para eliminar los elementos que están por encima de la diagonal principal.
3.  **Matriz diagonal D**: La matriz D obtenida de esta manera tendrá por diagonal la misma que ~A, es decir, los elementos ~a11, ~a22, ..., ~ann.

El c

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [36]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(
    max_results=5,        # cuántos resultados traer
    search_depth="basic", # lo mantenemos básico
)

tavily_tool.name = "tavily_search"
tavily_tool.description = (
    "Motor de búsqueda web en tiempo real. Úsalo para noticias, información reciente "
    "o datos que no estén en Wikipedia."
)


#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [37]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

wiki_wrapper = WikipediaAPIWrapper(
    lang="es",         # Wikipedia en español
    top_k_results=3,   # máximo 3 artículos relevantes
    doc_content_chars_max=4000,
)

wikipedia_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

wikipedia_tool.name = "wikipedia_search"
wikipedia_tool.description = (
    "Consulta a Wikipedia en español. Úsalo para conocimiento enciclopédico, "
    "definiciones, biografías y temas generales."
)


#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

In [38]:
import json

# Cliente de Gemini
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])


def decidir_tool(pregunta: str) -> str:
    """
    Heurística simple:
    - Tavily: preguntas sobre noticias, 'últimas', años recientes, 'hoy', 'reciente', 'noticias', etc.
    - Wikipedia: preguntas enciclopédicas (por defecto).
    """
    p = pregunta.lower()
    # señales de "tema reciente / noticias"
    if any(
        kw in p
        for kw in ["últimas", "ultimas", "hoy", "ayer", "reciente", "noticias", "2023", "2024", "2025"]
    ):
        return "tavily"
    # por defecto, enciclopédico
    return "wikipedia"


def agente_busqueda(pregunta: str) -> str:
    """
    Agente que decide qué tool usar (Tavily o Wikipedia),
    usa la tool para obtener contexto, y luego llama a Gemini
    para generar la respuesta final en español.
    """
    tool = decidir_tool(pregunta)
    print(f"[AGENTE] Pregunta: {pregunta}")
    print(f"[AGENTE] Tool elegida: {tool}")

    if tool == "tavily":
        # Convertimos el resultado a JSON formateado para mejor legibilidad
        raw_result = tavily_tool.run(pregunta)
        context = json.dumps(raw_result, ensure_ascii=False, indent=2)
    elif tool == "wikipedia":
        context = wikipedia_tool.run(pregunta)
    else:
        context = ""

    contents = [
        "Eres un asistente que responde SIEMPRE en español.",
        "A continuación tienes contexto obtenido con una herramienta externa (web o Wikipedia).",
        "Responde la pregunta usando ese contexto. Si el contexto no es suficiente, dilo explícitamente.",
        f"Contexto:\n{context}",
        f"Pregunta: {pregunta}",
        "Respuesta (en español):",
    ]

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents,
    )
    return response.text.strip()



#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

In [39]:
# Definimos algunas preguntas de prueba para el agente
preguntas_prueba = [
    "¿Quién es Gabriel Boric?",
    "¿Qué es el teorema de Pitágoras?",
    "¿Cuáles han sido las últimas noticias sobre la inflación en Chile 2024?",
]

# Ejecutamos el agente para cada pregunta de prueba
for q in preguntas_prueba:
    print("\n" + "=" * 100)
    respuesta = agente_busqueda(q)
    print("RESPUESTA DEL AGENTE:\n")
    print(respuesta)



[AGENTE] Pregunta: ¿Quién es Gabriel Boric?
[AGENTE] Tool elegida: wikipedia
RESPUESTA DEL AGENTE:

Gabriel Boric Font es un político chileno, egresado de Derecho, nacido en Punta Arenas el 11 de febrero de 1986. Actualmente se desempeña como presidente de la República de Chile desde el 11 de marzo de 2022, cargo que ocupará hasta el 11 de marzo de 2026.

Previamente, fue diputado de la República durante dos períodos (2014-2018 y 2018-2022) y presidente de la Federación de Estudiantes de la Universidad de Chile (FECh) entre 2011 y 2012, siendo uno de los líderes de las movilizaciones estudiantiles de ese año. Participó en la creación del Frente Amplio y fundó Convergencia Social.

Fue electo presidente en la segunda vuelta de las elecciones de 2021, obteniendo el 55,8% de los votos. Es el presidente con el mayor número de votos en la historia de Chile, el gobernante más joven del mundo en ejercicio al asumir, el presidente más joven en la historia de su país, el primero nacido después

El agente debería usar Tavily para datos recientes y wikipedia para datos más históricos.

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

In [40]:
from langchain_core.tools import Tool

# Tool para la solución RAG de los apuntes
rag_tool = Tool(
    name="rag_apuntes_matematicas",
    description=(
        "Usa los apuntes de MA1001, MA1002, MA1101 y MA1102 para responder "
        "preguntas de análisis y álgebra, usando RAG."
    ),
    func=lambda question: rag_chain(question, k=5),
)

# Tool para el agente de búsqueda (Tavily + Wikipedia)
agente_web_tool = Tool(
    name="agente_web_tavily_wikipedia",
    description=(
        "Usa Wikipedia y Tavily para responder preguntas generales, biográficas "
        "y de noticias actuales."
    ),
    func=lambda question: agente_busqueda(question),
)


#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

In [41]:
def supervisor(pregunta: str) -> str:
    """
    Agente supervisor: decide si asignar la pregunta al RAG de apuntes
    o al agente de búsqueda web (Tavily+Wikipedia).
    """
    p = pregunta.lower()

    # Heurística simple para reconocer preguntas "de curso" (apuntes)
    palabras_mate = [
        "sucesión", "sucesiones", "límite", "limite",
        "derivada", "integral", "matriz", "matrices",
        "vector", "espacio vectorial", "serie",
        "conjunto numerable", "ma1001", "ma1002", "ma1101", "ma1102",
    ]

    if any(kw in p for kw in palabras_mate):
        herramienta = rag_tool
    else:
        herramienta = agente_web_tool

    print(f"[SUPERVISOR] Pregunta: {pregunta}")
    print(f"[SUPERVISOR] Tool seleccionada: {herramienta.name}")

    # Usamos invoke para ejecutar la tool elegida
    return herramienta.invoke(pregunta)


#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

In [42]:
# Preguntas de la sección 2.1.4 (RAG sobre apuntes)
preguntas_rag = [
    "¿Cómo se define una matriz A de m filas y n columnas con coeficientes en un cuerpo K?",
    "¿Cuál es la definición de subsucesión de una sucesión (s_n)?",
    "¿Qué significa que un conjunto sea numerable?",
]

# Preguntas de la sección 2.2.4 (agente web)
preguntas_agente = [
    "¿Quién es Gabriel Boric?",
    "¿Qué es el teorema de Pitágoras?",
    "¿Cuáles han sido las últimas noticias sobre la inflación en Chile 2024?",
]

for q in preguntas_rag + preguntas_agente:
    print("\n" + "=" * 120)
    print("PREGUNTA:", q)
    respuesta = supervisor(q)
    print("\nRESPUESTA DEL SUPERVISOR:\n")
    print(respuesta)



PREGUNTA: ¿Cómo se define una matriz A de m filas y n columnas con coeficientes en un cuerpo K?
[SUPERVISOR] Pregunta: ¿Cómo se define una matriz A de m filas y n columnas con coeficientes en un cuerpo K?
[SUPERVISOR] Tool seleccionada: rag_apuntes_matematicas

RESPUESTA DEL SUPERVISOR:

Una matriz A de m filas y n columnas con coeficientes en un cuerpo K se denota como A = (aij), donde aij son los coeficientes en K. El conjunto de todas estas matrices se denomina Mmn(K).

PREGUNTA: ¿Cuál es la definición de subsucesión de una sucesión (s_n)?
[SUPERVISOR] Pregunta: ¿Cuál es la definición de subsucesión de una sucesión (s_n)?
[SUPERVISOR] Tool seleccionada: rag_apuntes_matematicas

RESPUESTA DEL SUPERVISOR:

Una matriz A de m filas y n columnas con coeficientes en un cuerpo K se denota como A = (aij), donde aij son los coeficientes en K. El conjunto de todas estas matrices se denomina Mmn(K).

PREGUNTA: ¿Cuál es la definición de subsucesión de una sucesión (s_n)?
[SUPERVISOR] Pregunta:

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [43]:
%pip install --upgrade --quiet gradio

Note: you may need to restart the kernel to use updated packages.


Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [44]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

TypeError: ChatInterface.__init__() got an unexpected keyword argument 'type'

# Conclusión
Éxito!
<center>
<img src ="https://media.tenor.com/MRQgxcelAV8AAAAM/perry-the-platypus-phineas-and-ferb.gif" width = 400 />